# Phylogeny of *Muscari* using genomic ddRAD data

In [1]:
## conda install ipyrad -c ipyrad
## conda install toytree -c eaton-lab
## conda install sra-tools -c bioconda
## conda install entrez-direct -c bioconda

In [2]:
## import
import ipyrad as ip
import ipyrad.analysis as ipa
import ipyparallel as ipp
import toytree
import toyplot

## print Version of ipyrad und toytree
print("ipyrad v. {}".format(ip.__version__))
print("toytree v. {}".format(toytree.__version__))

## print Version of Python
from platform import python_version
print("Python v.", python_version())

ipyrad v. 0.9.64
toytree v. 2.0.5
Python v. 3.7.9


#### Parallel processes on independent Python kernels
To start a parallel client you must run the command-line program 'ipcluster'. This will essentially start a number of independent Python processes (kernels) which we can then send bits of work to do. The cluster can be stopped and restarted independently of this notebook, which is convenient for working on a cluster where connecting to many cores is not always immediately available.

Open a terminal and type the following command to start an ipcluster instance with N engines.

In [3]:
## ipcluster start --n=16

In [48]:
## connect to cluster
ipyclient = ipp.Client()
print(ip.cluster_info(ipyclient))

Parallel connection | Cryptantha: 64 cores
None


## Data Assembly
### Create an Assembly object and modify *ipyrad* params file
This object stores the parameters of the assembly and the organization of the data

In [6]:
## Provide a name for the assembly
data = ip.Assembly("Muscari")

New Assembly: Muscari


In [7]:
## set parameters
data.set_params("project_dir", "Mus_Assembly")
data.set_params("sorted_fastq_path", "./Mus_fastq/*.fastq.gz")
data.set_params("clust_threshold", "0.85")
data.set_params("max_Hs_consens", (0.05))
data.set_params("restriction_overhang", ('TGCAG', 'GGCC'))
data.set_params("output_formats", "*")
data.set_params("datatype", "ddrad")

## see / print all parameters
data.get_params()

0   assembly_name               Muscari                                      
1   project_dir                 ./Mus_Assembly                               
2   raw_fastq_path                                                           
3   barcodes_path                                                            
4   sorted_fastq_path           ./Mus_fastq/*.fastq.gz                       
5   assembly_method             denovo                                       
6   reference_sequence                                                       
7   datatype                    ddrad                                        
8   restriction_overhang        ('TGCAG', 'GGCC')                            
9   max_low_qual_bases          5                                            
10  phred_Qscore_offset         33                                           
11  mindepth_statistical        6                                            
12  mindepth_majrule            6                               

### Assemble the data from step 1 to 6

In [8]:
## run step 1 to 6 of the assembly
data.run("123456", force = True)

Parallel connection | Cryptantha: 64 cores
[####################] 100% 0:00:03 | loading reads        | s1 |
[####################] 100% 0:00:23 | processing reads     | s2 |
[####################] 100% 0:00:04 | dereplicating        | s3 |
[####################] 100% 0:06:55 | clustering/mapping   | s3 |
[####################] 100% 0:00:01 | building clusters    | s3 |
[####################] 100% 0:00:00 | chunking clusters    | s3 |
[####################] 100% 0:03:57 | aligning clusters    | s3 |
[####################] 100% 0:00:15 | concat clusters      | s3 |
[####################] 100% 0:00:01 | calc cluster stats   | s3 |
[####################] 100% 0:00:15 | inferring [H, E]     | s4 |
[####################] 100% 0:00:01 | calculating depths   | s5 |
[####################] 100% 0:00:01 | chunking clusters    | s5 |
[####################] 100% 0:01:53 | consens calling      | s5 |
[####################] 100% 0:00:02 | indexing alleles     | s5 |
[####################] 100% 0:00:

In [9]:
#data.stats.sort_values(by=['reads_consens'])
data.stats

,state,reads_raw,reads_passed_filter,clusters_total,clusters_hidepth,hetero_est,error_est,reads_consens
Bellevalia_dubia_W6083,6,1030736,1029284,95294,20981,0.013,0.003,19338
Bellevalia_paradoxa_ED1272,6,1636142,1634727,108498,30267,0.013,0.003,27749
Bellevalia_speciosa_W6085,6,1416391,1414294,95536,25347,0.016,0.003,22957
Brimeura_amethystina_W6084,6,1554459,1551802,424844,28296,0.034,0.005,20711
Leopoldia_caucasica_ED1262,6,1462581,1461153,77305,22469,0.013,0.003,20653
Leopoldia_comosa_ED1256,6,1299389,1298312,90402,25831,0.015,0.003,23363
Leopoldia_comosa_ED1274,6,1464810,1463759,90898,24322,0.015,0.002,22075
Leopoldia_comosa_ED3539,6,2065757,2064748,368808,46895,0.013,0.003,42303
Leopoldia_comosa_ED3965,6,1232250,1231244,94455,25479,0.013,0.003,23472
Leopoldia_cycladica_W6082,6,1664161,1661171,152200,36680,0.025,0.003,30683


In [10]:
## show assemby stats until step 6
#data.stats
data.stats.sort_values(by = ['reads_consens'])

,state,reads_raw,reads_passed_filter,clusters_total,clusters_hidepth,hetero_est,error_est,reads_consens
Muscari_anatolicum_W6087,6,1224126,1221220,56984,18627,0.015,0.003,17017
Muscari_adilii_W6090,6,1234147,1231484,117636,19475,0.018,0.003,17235
Muscari_pulchellum_ED3231,6,1007408,1006385,69798,19257,0.013,0.003,17715
Muscari_armeniacum_W6089,6,1436401,1433387,83343,19937,0.018,0.003,17821
Muscari_mirum_ED1250,6,1147548,1146637,80280,20292,0.013,0.002,18748
Bellevalia_dubia_W6083,6,1030736,1029284,95294,20981,0.013,0.003,19338
Pseudomuscari_coeruleum_ED1261,6,1533268,1532125,93250,21469,0.012,0.002,19834
Leopoldia_tenuiflora_ED1263,6,983193,982239,73834,21497,0.009,0.003,20148
Leopoldia_caucasica_ED1262,6,1462581,1461153,77305,22469,0.013,0.003,20653
Leopoldia_neumannii_ED1243,6,1022622,1022000,74881,21887,0.008,0.003,20671


## Final assembly with different `min_samples_locus` settings for different analysis

1. Phylogenetic analysis 
    - RAxML
    - MrBayes
    - tetRAD
2. Population analysis of selected clades
    - PCA
    - STRUCTURE
    - TreeMix
3. Test for introgression using abba-baba test
    - ...
    
#### In case comming back to continue from here, load assembly object to continue after step 6

In [6]:
## load assembly object when comming back
data = ip.load_json("./Mus_Assembly/Muscari.json")

## check again the stat-file sorted by number of consensus reads
#data.stats.sort_values(by=['reads_consens'])

## check name
#data.stats

loading Assembly: Muscari
from saved path: ~/GBS/Muscari/Mus_Assembly/Muscari.json


### 1. Assembly for Phylogenetic analysis
#### *But first lets exclude samples with low read number (< 1000 reads after step 6), which are outsite the target group or with odd placements in preliminary analysis:*

**Samples outsite the target group are:**
- ...

In [ ]:
## exclude samples from assembly with ...
keep_list = [i for i in data.samples.keys() if i not in [
    ## ... low read number (< 5000 )
    #"", "",
    
    ## ... other samples to exclude
    "", "", "",
]]

## make a new data branch from the keep_list
data = data.branch("data", subsamples = keep_list, force = True)

## double check taxon sampling
#data.stats.sort_values(by=['reads_consens'])
data.stats

In [ ]:
################################################################
#############    TEMPLATE :::: do not run    ###################
################################################################

## ::: Template for step 7 assembly with in- and outgroup ::: ##
## create a branch for outputs with min_samples = x
min4 = data.branch("min_4")
min4.set_params("min_samples_locus", 4)
min4.run("7")

## ::: Template for step 7 assembly with in- and outgroup ::: ##
## create a branch for outputs with min_samples = x BUT only for ingroup
pops = data.branch("pops")
pops.population = {
    "ingroup": (20, [i for i in pops.samples if "Frai" in i]),
    "outgroup": (0, [i for i in pops.samples if "Frai" not in i])
}
pops.run("7", force = True)

################################################################
#############    TEMPLATE :::: do not run    ###################
################################################################

In [11]:
## ::::::: WORK IN PROGRESS
## WRITE THE RESULTS OF THE PERCENTAGE LOOP INTO A DICTIONARY 
## WHICH THEN CAN BE USED IN THE FOLLOWING STEPS
## INSTEAD OF MAKING THE DICTIONARY BY HAND 


## first check number of remaining samples
ingroup = data.stats.state.count() - 4
print("Number of ingroup taxa:", ingroup)
print("Calculate different sets of missing data:")

## for loop to calculate different values for min_sample_locus
percent = [10, 15, 20, 25, 30, 35, 40]
for i in percent:
    res = ingroup / 100 * i
    print(i,"% = ", round(res))

Number of ingroup taxa: 40
Calculate different sets of missing data:
10 % =  4
15 % =  6
20 % =  8
25 % =  10
30 % =  12
35 % =  14
40 % =  16


In [12]:
## Run the final assembly step 7 through for loop with different min_sample_locus
## based on estimated number of remaining samples MINUS outgroup

## make a dictionary with the percentage of missing data as keys and 
## the actual min_sample_locus specified as values based on the number of "ingroup samples"
sample_dict = {10: 4,
               15: 6,
               20: 8,
               25: 10,
               30: 12,
               35: 14,
               40: 16}

## define list with ingroup wildcards
#ingroup = ["Mus", "Pseu", "Leop"]

## loop over the dictionary 
for key, value in sample_dict.items():
    newname = "pops_{}".format(key)
    newdata = data.branch(newname)
    newdata.populations = {
        "ingroup":  (value, [i for i in newdata.samples if "B" not in i]),
        "outgroup": (0,     [i for i in newdata.samples if "B" in i]),
         }
    
    newdata.run("7", force = True)

Parallel connection | Cryptantha: 64 cores
[####################] 100% 0:00:05 | applying filters     | s7 |
[####################] 100% 0:00:17 | building arrays      | s7 |
[####################] 100% 0:00:07 | writing conversions  | s7 |
[####################] 100% 0:00:18 | indexing vcf depths  | s7 |
[####################] 100% 0:00:50 | writing vcf output   | s7 |
Parallel connection | Cryptantha: 64 cores
[####################] 100% 0:00:02 | applying filters     | s7 |
[####################] 100% 0:00:11 | building arrays      | s7 |
[####################] 100% 0:00:04 | writing conversions  | s7 |
[####################] 100% 0:00:06 | indexing vcf depths  | s7 |
[####################] 100% 0:00:31 | writing vcf output   | s7 |
Parallel connection | Cryptantha: 64 cores
[####################] 100% 0:00:02 | applying filters     | s7 |
[####################] 100% 0:00:08 | building arrays      | s7 |
[####################] 100% 0:00:03 | writing conversions  | s7 |
[############

In [ ]:
## Does the same as above but without ingroup and outgroup
sample_dict = {10: 4,
               15: 6,
               20: 8,
               25: 10,
               30: 11,
               35: 13,
               40: 15}

## loop over the dictionary 
for key, value in sample_dict.items():
    newname = "min_{}".format(key)
    newdata = data.branch(newname)
    newdata.set_params("min_samples_locus", value)
    newdata.run("7", force = True)

## Phylogenetic downstream analysis
First, check if you need to install additional packages which are not included in the ipyrad package dependencies. Use the following commands to install the packages in the terminal.

In [ ]:
## following programs are required
# conda install toytree -c eaton-lab
# conda install tetrad -c eaton-lab -c conda-forge
# conda install raxml -c bioconda

### RAxML

In [ ]:
## create a raxml analysis object for the Backbone tree
rax = ipa.raxml(
    name = Cris_pops30.name,
    data = Cris_pops30.outfiles.phy,
    workdir = "./Mus_Analysis/Mus_RAxML",
    T = 16,
    N = 200,
    o = "Bellevallia_pycantha_ED1272",
    )

In [ ]:
## Plot the resulting tree

tre = toytree.tree("./Mus_Analysis/Mus_IQtree/pops_30.phy.contree")
rtre = tre.root(wildcard = "Belle")
#rtre.draw(tip_labels_align=True, node_labels="support")

# use canvas and axes function in order use export function
canvas, axes, mark = rtre.ladderize(1).draw(
    width = 1400,
    height = 900,
    use_edge_length = False,
    tip_labels_align = True,
    node_labels='support',
    node_sizes=0,
    node_labels_style={"font-size": "16px",
                       "baseline-shift": "7px",
                       "-toyplot-anchor-shift": "-13px"},
    );

In [60]:
## Plot all three RAxML trees together

## Load trees
tre15 = toytree.tree("/home/tim/GBS/Muscari/Mus_Analysis/Mus_RAxML/Mus_RAxML_20210802/RAxML_bipartitions.pops_15.phy")
tre20 = toytree.tree("/home/tim/GBS/Muscari/Mus_Analysis/Mus_RAxML/Mus_RAxML_20210802/RAxML_bipartitions.pops_20.phy")
tre25 = toytree.tree("/home/tim/GBS/Muscari/Mus_Analysis/Mus_RAxML/Mus_RAxML_20210802/RAxML_bipartitions.pops_25.phy")
tre30 = toytree.tree("/home/tim/GBS/Muscari/Mus_Analysis/Mus_RAxML/Mus_RAxML_20210802/RAxML_bipartitions.pops_30.phy")
tre35 = toytree.tree("/home/tim/GBS/Muscari/Mus_Analysis/Mus_RAxML/Mus_RAxML_20210802/RAxML_bipartitions.pops_35.phy")
tre40 = toytree.tree("/home/tim/GBS/Muscari/Mus_Analysis/Mus_RAxML/Mus_RAxML_20210802/RAxML_bipartitions.pops_40.phy")

tre15 = tre15.root(wildcard = "Brimeura")
tre20 = tre20.root(wildcard = "Brimeura")
tre25 = tre25.root(wildcard = "Brimeura")
tre30 = tre30.root(wildcard = "Brimeura")
tre35 = tre35.root(wildcard = "Brimeura")
tre40 = tre40.root(wildcard = "Brimeura")


## set dimensions of the canvas
canvas = toyplot.Canvas(width = 2000, height = 2000)

## dissect canvas into multiple cartesian areas (x1, x2, y1, y2)
ax0 = canvas.cartesian(bounds=('2%',  '30%', '5%',  '47.5%'))
ax1 = canvas.cartesian(bounds=('33%', '63%', '5%',  '47.5%'))
ax2 = canvas.cartesian(bounds=('66%', '96%', '5%',  '47.5%'))
ax3 = canvas.cartesian(bounds=('2%',  '30%', '50%', '97.5%'))
ax4 = canvas.cartesian(bounds=('33%', '63%', '50%', '97.5%'))
ax5 = canvas.cartesian(bounds=('66%', '96%', '50%', '97.5%'))

# call draw with the 'axes' argument to pass it to a specific cartesian area
style = {
    "tip_labels_align": True,
    "tip_labels_style": {"font-size": "11px"},
    "node_labels_style":{"font-size": "12px",
                        "baseline-shift": "7px",
                        "-toyplot-anchor-shift": "-13px"},
}
tre15.ladderize(1).draw(
    axes = ax0,
    **style,
    node_sizes = 0,
    node_labels = 'support');

tre20.ladderize(1).draw(
    axes = ax1,
    **style,
    node_sizes = 0,
    node_labels = 'support');

tre25.ladderize(1).draw(
    axes = ax2,
    **style,
    node_sizes = 0,
    node_labels = 'support');

tre30.ladderize(1).draw(
    axes = ax3,
    **style,
    node_sizes = 0,
    node_labels = 'support');

tre35.ladderize(1).draw(
    axes = ax4,
    **style,
    node_sizes = 0,
    node_labels = 'support');

tre40.ladderize(1).draw(
    axes = ax5,
    **style,
    node_sizes = 0,
    node_labels = 'support');

# hide the axes (e.g, ticks and splines)
ax0.show = False; ax1.show = False; ax2.show = False;
ax3.show = False; ax4.show = False; ax5.show = False;

<svg class="toyplot-canvas-Canvas" height="2000.0px" id="t7bdd3c1095a4441c954e0d80b6e30aef" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 2000.0 2000.0" width="2000.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 99 95 86 100 100 100 100 76 100 100 100 100 77 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 Brimeura_amethystina_W6084 Bellevalia_paradoxa_ED1272 Bellevalia_dubia_W6083 Bellevalia_speciosa_W6085 Muscari_macrocarpum_ED1252 Muscari_racemosum_ED1258 Pseudomuscari_azureum_ED1270 Pseudomuscari_chalusicum_ED1255 Pseudomuscari_inconstrictum_ED3234 Muscari_parviflorum_ED1245 Muscari_commutatum_ED3538 Muscari_sivrihisardaghlarensis_ED1278 Muscari_anatolicum_W6087 Muscari_vularlii_ED3232 Muscari_discolor_ED1266 Pseudomuscari_pallens_ED1267 Pseudomuscari_coeruleum_ED1261 Muscari_adilii_W6090 Muscari_armeniacum_W6089 Muscari_armeniacum_ED1244 Muscari_neglectum_ED1253 Muscari_baeticum_ED1281 Muscari_neglectum_ED1254 Muscari_botryoides_ED1279 Muscari_pulchellum_ED3231 Muscari_kerkis_ED1280 Muscari_bourgaei_ED1259 Muscari_latifolium_ED1265 Leopoldia_tenuiflora_ED1263 Muscari_massayanum_ED1251 Leopoldia_longipes_ED3233 Leopoldia_neumannii_ED1607 Leopoldia_neumannii_ED1243 Muscari_mirum_ED1250 Leopoldia_caucasica_ED1262 Leopoldia_matritensis_ED1282 Leopoldia_comosa_ED3539 Leopoldia_comosa_ED1274 Leopoldia_comosa_ED3965 Leopoldia_comosa_ED1256 Leopoldia_weissii_ED1608 Leopoldia_spreitzenhoferi_ED1248 Leopoldia_weissii_W6081 Leopoldia_cycladica_W6082 50 89 47 100 99 100 100 52 100 100 100 100 82 95 100 100 100 100 100 100 100 100 100 100 99 100 100 100 100 100 100 100 99 100 100 100 100 100 100 100 100 100 Brimeura_amethystina_W6084 Bellevalia_paradoxa_ED1272 Bellevalia_dubia_W6083 Bellevalia_speciosa_W6085 Muscari_racemosum_ED1258 Muscari_macrocarpum_ED1252 Muscari_parviflorum_ED1245 Pseudomuscari_inconstrictum_ED3234 Pseudomuscari_chalusicum_ED1255 Pseudomuscari_azureum_ED1270 Muscari_commutatum_ED3538 Muscari_sivrihisardaghlarensis_ED1278 Muscari_anatolicum_W6087 Muscari_vularlii_ED3232 Muscari_discolor_ED1266 Pseudomuscari_pallens_ED1267 Pseudomuscari_coeruleum_ED1261 Muscari_adilii_W6090 Muscari_armeniacum_ED1244 Muscari_armeniacum_W6089 Muscari_neglectum_ED1253 Muscari_baeticum_ED1281 Muscari_neglectum_ED1254 Muscari_botryoides_ED1279 Muscari_pulchellum_ED3231 Muscari_kerkis_ED1280 Muscari_bourgaei_ED1259 Muscari_latifolium_ED1265 Leopoldia_tenuiflora_ED1263 Muscari_massayanum_ED1251 Leopoldia_longipes_ED3233 Leopoldia_neumannii_ED1607 Leopoldia_neumannii_ED1243 Muscari_mirum_ED1250 Leopoldia_caucasica_ED1262 Leopoldia_matritensis_ED1282 Leopoldia_comosa_ED3539 Leopoldia_comosa_ED1274 Leopoldia_comosa_ED3965 Leopoldia_comosa_ED1256 Leopoldia_weissii_ED1608 Leopoldia_spreitzenhoferi_ED1248 Leopoldia_cycladica_W6082 Leopoldia_weissii_W6081 88 84 82 99 100 100 99 88 100 96 98 100 82 100 100 100 86 100 100 100 100 100 100 100 96 100 100 100 100 100 100 100 100 100 100 100 100 100 100 99 100 100 Brimeura_amethystina_W6084 Bellevalia_paradoxa_ED1272 Bellevalia_dubia_W6083 Bellevalia_speciosa_W6085 Muscari_macrocarpum_ED1252 Muscari_racemosum_ED1258 Pseudomuscari_chalusicum_ED1255 Pseudomuscari_azureum_ED1270 Muscari_parviflorum_ED1245 Pseudomuscari_inconstrictum_ED3234 Muscari_commutatum_ED3538 Muscari_sivrihisardaghlarensis_ED1278 Muscari_vularlii_ED3232 Muscari_anatolicum_W6087 Muscari_discolor_ED1266 Pseudomuscari_coeruleum_ED1261 Pseudomuscari_pallens_ED1267 Muscari_adilii_W6090 Muscari_armeniacum_W6089 Muscari_armeniacum_ED1244 Muscari_neglectum_ED1253 Muscari_baeticum_ED1281 Muscari_botryoides_ED1279 Muscari_neglectum_ED1254 Musc

In [61]:
import toyplot.pdf
toyplot.pdf.render(canvas, "/home/tim/GBS/Muscari/Mus_Analysis/Mus_RAxML/RAxML_Figures/Muscari_20210802_15-20-25-30-35-40_NoRoot.pdf");

### tetRAD
##### run a single tetRAD analysis

In [36]:
# the path to your sequence data in HDF5 format
data = "/home/tim/GBS/Muscari/Mus_Assembly/pops_15_outfiles/pops_15.snps.hdf5"

In [12]:
# init analysis object with input data and (optional) parameter options
tet = ipa.tetrad(
    name = "Mus_pops_15",
    data = data,
    workdir = "./Mus_Analysis/Mus_tetRAD",
    nquartets = 1e6,
    nboots = 200,
)

loading snps array [44 taxa x 114197 snps]
max unlinked SNPs per quartet [nloci]: 14705
quartet sampler [full]: 135751 / 135751


In [13]:
tet.run(auto = True, force = True)

Parallel connection | Cryptantha: 64 cores
initializing quartet sets database
[####################] 100% 0:00:07 | full tree * | avg SNPs/qrt: 1014 
[####################] 100% 0:00:04 | boot rep. 1 | avg SNPs/qrt: 1017 
Keyboard Interrupt by user



###### run multiple retRAD analysis in a for loop

In [ ]:
dict = {
    "pop15": "/home/tim/GBS/Muscari/Mus_Assembly/pops_15_outfiles/pops_15.snps.hdf5",
    "pop20": "/home/tim/GBS/Muscari/Mus_Assembly/pops_20_outfiles/pops_20.snps.hdf5",
    "pop25": "/home/tim/GBS/Muscari/Mus_Assembly/pops_25_outfiles/pops_25.snps.hdf5",
    "pop30": "/home/tim/GBS/Muscari/Mus_Assembly/pops_30_outfiles/pops_30.snps.hdf5",
    "pop35": "/home/tim/GBS/Muscari/Mus_Assembly/pops_35_outfiles/pops_35.snps.hdf5",
    "pop40": "/home/tim/GBS/Muscari/Mus_Assembly/pops_40_outfiles/pops_40.snps.hdf5"
}

In [59]:
#test = [pops15, pops20, pops25, pops30, pops35, pops40]

for key, value in dict.items():
    tet = ipa.tetrad(
        name = "Mus_tet_" + str(key),
        data = value,
        workdir = "./Mus_Analysis/Mus_tetRAD",
        nquartets = 1e6,
        nboots = 200)
    ## run 
    tet.run(auto = True, force = True)

loading snps array [44 taxa x 114197 snps]
max unlinked SNPs per quartet [nloci]: 14705
quartet sampler [full]: 135751 / 135751
Parallel connection | Cryptantha: 64 cores
initializing quartet sets database
[####################] 100% 0:00:07 | full tree * | avg SNPs/qrt: 1014 
[####################] 100% 0:00:04 | boot rep. 1 | avg SNPs/qrt: 980 
[####################] 100% 0:00:04 | boot rep. 2 | avg SNPs/qrt: 1061 
[####################] 100% 0:00:04 | boot rep. 3 | avg SNPs/qrt: 1000 
[####################] 100% 0:00:04 | boot rep. 4 | avg SNPs/qrt: 978 
[####################] 100% 0:00:04 | boot rep. 5 | avg SNPs/qrt: 1004 
[####################] 100% 0:00:04 | boot rep. 6 | avg SNPs/qrt: 998 
[####################] 100% 0:00:04 | boot rep. 7 | avg SNPs/qrt: 1008 
[####################] 100% 0:00:04 | boot rep. 8 | avg SNPs/qrt: 1025 
[####################] 100% 0:00:04 | boot rep. 9 | avg SNPs/qrt: 1036 
[####################] 100% 0:00:03 | boot rep. 10 | avg SNPs/qrt: 1026 
[###

In [64]:
## Plot all six tetRAD coalescent trees together

## Load trees
tet15 = toytree.tree("./Mus_Analysis/Mus_tetRAD/Mus_tet_pop15.tree.cons").root(wildcard = "Brimeura")
tet20 = toytree.tree("./Mus_Analysis/Mus_tetRAD/Mus_tet_pop20.tree.cons").root(wildcard = "Brimeura")
tet25 = toytree.tree("./Mus_Analysis/Mus_tetRAD/Mus_tet_pop25.tree.cons").root(wildcard = "Brimeura")
tet30 = toytree.tree("./Mus_Analysis/Mus_tetRAD/Mus_tet_pop30.tree.cons").root(wildcard = "Brimeura")
tet35 = toytree.tree("./Mus_Analysis/Mus_tetRAD/Mus_tet_pop35.tree.cons").root(wildcard = "Brimeura")
tet40 = toytree.tree("./Mus_Analysis/Mus_tetRAD/Mus_tet_pop40.tree.cons").root(wildcard = "Brimeura")

## set dimensions of the canvas
canvas = toyplot.Canvas(width = 2000, height = 2000)

## dissect canvas into multiple cartesian areas (x1, x2, y1, y2)
ax0 = canvas.cartesian(bounds=('2%',  '30%', '5%',  '47.5%'))
ax1 = canvas.cartesian(bounds=('33%', '63%', '5%',  '47.5%'))
ax2 = canvas.cartesian(bounds=('66%', '96%', '5%',  '47.5%'))
ax3 = canvas.cartesian(bounds=('2%',  '30%', '50%', '97.5%'))
ax4 = canvas.cartesian(bounds=('33%', '63%', '50%', '97.5%'))
ax5 = canvas.cartesian(bounds=('66%', '96%', '50%', '97.5%'))

# call draw with the 'axes' argument to pass it to a specific cartesian area
style = {
    "tip_labels_align": True,
    "tip_labels_style": {"font-size": "11px"},
    "node_labels_style":{"font-size": "12px",
                        "baseline-shift": "7px",
                        "-toyplot-anchor-shift": "-13px"},
}
tet15.ladderize(1).draw(
    axes = ax0,
    **style,
    node_sizes = 0,
    node_labels = 'support');

tet20.ladderize(1).draw(
    axes = ax1,
    **style,
    node_sizes = 0,
    node_labels = 'support');

tet25.ladderize(1).draw(
    axes = ax2,
    **style,
    node_sizes = 0,
    node_labels = 'support');

tet30.ladderize(1).draw(
    axes = ax3,
    **style,
    node_sizes = 0,
    node_labels = 'support');

tet35.ladderize(1).draw(
    axes = ax4,
    **style,
    node_sizes = 0,
    node_labels = 'support');

tet40.ladderize(1).draw(
    axes = ax5,
    **style,
    node_sizes = 0,
    node_labels = 'support');

# hide the axes (e.g, ticks and splines)
ax0.show = False; ax1.show = False; ax2.show = False;
ax3.show = False; ax4.show = False; ax5.show = False;

<svg class="toyplot-canvas-Canvas" height="2000.0px" id="t588f44e0745a48d692a28abd1799eda7" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 2000.0 2000.0" width="2000.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 58 40 44 62 58 42 100 49 100 55 41 100 64 62 99 100 70 96 100 100 100 99 100 98 58 100 87 100 100 100 100 99 62 90 100 92 100 99 84 89 100 100 Brimeura_amethystina_W6084 Bellevalia_paradoxa_ED1272 Bellevalia_dubia_W6083 Bellevalia_speciosa_W6085 Muscari_macrocarpum_ED1252 Muscari_racemosum_ED1258 Pseudomuscari_azureum_ED1270 Pseudomuscari_chalusicum_ED1255 Pseudomuscari_inconstrictum_ED3234 Muscari_parviflorum_ED1245 Muscari_commutatum_ED3538 Muscari_sivrihisardaghlarensis_ED1278 Muscari_anatolicum_W6087 Muscari_vularlii_ED3232 Muscari_discolor_ED1266 Pseudomuscari_coeruleum_ED1261 Pseudomuscari_pallens_ED1267 Muscari_adilii_W6090 Muscari_armeniacum_ED1244 Muscari_armeniacum_W6089 Muscari_neglectum_ED1253 Muscari_baeticum_ED1281 Muscari_neglectum_ED1254 Muscari_botryoides_ED1279 Muscari_pulchellum_ED3231 Muscari_kerkis_ED1280 Muscari_latifolium_ED1265 Muscari_bourgaei_ED1259 Leopoldia_tenuiflora_ED1263 Muscari_massayanum_ED1251 Leopoldia_longipes_ED3233 Leopoldia_neumannii_ED1607 Leopoldia_neumannii_ED1243 Muscari_mirum_ED1250 Leopoldia_matritensis_ED1282 Leopoldia_caucasica_ED1262 Leopoldia_spreitzenhoferi_ED1248 Leopoldia_weissii_ED1608 Leopoldia_weissii_W6081 Leopoldia_cycladica_W6082 Leopoldia_comosa_ED3539 Leopoldia_comosa_ED3965 Leopoldia_comosa_ED1256 Leopoldia_comosa_ED1274 38 58 60 50 50 100 37 56 100 52 35 100 54 54 100 100 65 96 100 99 100 100 100 97 54 100 92 100 100 100 100 100 64 94 100 89 100 96 82 88 100 100 Brimeura_amethystina_W6084 Bellevalia_paradoxa_ED1272 Bellevalia_dubia_W6083 Bellevalia_speciosa_W6085 Muscari_macrocarpum_ED1252 Muscari_racemosum_ED1258 Pseudomuscari_azureum_ED1270 Pseudomuscari_chalusicum_ED1255 Muscari_parviflorum_ED1245 Pseudomuscari_inconstrictum_ED3234 Muscari_commutatum_ED3538 Muscari_sivrihisardaghlarensis_ED1278 Muscari_anatolicum_W6087 Muscari_vularlii_ED3232 Muscari_discolor_ED1266 Pseudomuscari_coeruleum_ED1261 Pseudomuscari_pallens_ED1267 Muscari_adilii_W6090 Muscari_armeniacum_ED1244 Muscari_armeniacum_W6089 Muscari_neglectum_ED1253 Muscari_baeticum_ED1281 Muscari_botryoides_ED1279 Muscari_neglectum_ED1254 Muscari_pulchellum_ED3231 Muscari_kerkis_ED1280 Muscari_bourgaei_ED1259 Muscari_latifolium_ED1265 Leopoldia_tenuiflora_ED1263 Muscari_massayanum_ED1251 Leopoldia_longipes_ED3233 Leopoldia_neumannii_ED1607 Leopoldia_neumannii_ED1243 Muscari_mirum_ED1250 Leopoldia_matritensis_ED1282 Leopoldia_caucasica_ED1262 Leopoldia_comosa_ED3539 Leopoldia_comosa_ED3965 Leopoldia_comosa_ED1274 Leopoldia_comosa_ED1256 Leopoldia_spreitzenhoferi_ED1248 Leopoldia_weissii_ED1608 Leopoldia_cycladica_W6082 Leopoldia_weissii_W6081 38 62 57 46 100 55 37 56 100 50 37 100 63 65 99 100 68 93 100 96 100 100 100 100 56 100 84 100 99 100 97 100 53 86 100 87 100 98 85 86 100 100 Brimeura_amethystina_W6084 Bellevalia_paradoxa_ED1272 Bellevalia_dubia_W6083 Bellevalia_speciosa_W6085 Muscari_macrocarpum_ED1252 Muscari_racemosum_ED1258 Pseudomuscari_inconstrictum_ED3234 Muscari_parviflorum_ED1245 Pseudomuscari_chalusicum_ED1255 Pseudomuscari_azureum_ED1270 Muscari_commutatum_ED3538 Muscari_sivrihisardaghlarensis_ED1278 Muscari_anatolicum_W6087 Muscari_vularlii_ED3232 Muscari_discolor_ED1266 Pseudomuscari_pallens_ED1267 Pseudomuscari_coeruleum_ED1261 Muscari_adilii_W6090 Muscari_armeniacum_W6089 Muscari_armeniacum_ED1244 Muscari_neglectum_ED1253 Muscari_baeticum_ED1281 Muscari_neglectum_ED1254 Muscari_botryoides_ED1279 Muscari_pulchellum_ED3231 Muscari_kerkis_ED1280 Mus

In [65]:
import toyplot.pdf
toyplot.pdf.render(canvas, "/home/tim/GBS/Muscari/Mus_Analysis/Mus_tetRAD/tetRAD_Figures/Muscari_tet_20210802_15-20-25-30-35-40.pdf");

In [71]:
tet30 = toytree.tree("./Mus_Analysis/Mus_tetRAD/Mus_tet_pop30.tree.boots")
#tet30.treelist = [i.root("Brimeura") for i in tet30.treelist]
tet30.draw_cloud_tree(
    height=600,
    width=400,
    use_edge_lengths=False,
    html=True,
);



AttributeError: 'ToyTree' object has no attribute 'draw_cloud_tree'

In [ ]:
## Load trees
fulltree = toytree.tree("/home/tim/GBS/Muscari/analysis-tetrad/Mus_pops_30.tree").root(["Bellevallia_pycantha_ED1272"])
constree = toytree.tree("/home/tim/GBS/Muscari/analysis-tetrad/Mus_pops_30.tree.cons").root(["Bellevallia_pycantha_ED1272"])
#cloudtre = toytree.mtree(tet.trees.boots)
#cloudtre.treelist = [i.root(["Bellevallia_pycantha_ED1272"]) for i in cloudtre.treelist]

## set dimensions of the canvas
canvas = toyplot.Canvas(width = 1800, height = 700)

## dissect canvas into multiple cartesian areas (x1, x2, y1, y2)
ax0 = canvas.cartesian(bounds=('2%',  '30%', '5%',  '97.5%'))
ax1 = canvas.cartesian(bounds=('33%', '63%', '5%',  '97.5%'))
#ax2 = canvas.cartesian(bounds=('66%', '96%', '5%',  '97.5%'))

# call draw with the 'axes' argument to pass it to a specific cartesian area
style = {
    "tip_labels_align": True,
    "tip_labels_style": {"font-size": "12px"},
    "node_labels_style":{"font-size": "12px",
                        "baseline-shift": "7px",
                        "-toyplot-anchor-shift": "-13px"},
}
fulltree.ladderize(1).draw(
    axes = ax0,
    **style,
    node_sizes = 0,
    node_labels = fulltree.get_node_values("support"));

constree.ladderize(1).draw(
    axes = ax1,
    **style,
    node_sizes = 0,
    node_labels = constree.get_node_values("support"));

#cloudtre.draw_cloud_tree(
#    axes = ax2,
#    use_edge_lengths = False,
#    node_sizes = 0,
#    #node_labels = tre25.get_node_values("support")
#);


# hide the axes (e.g, ticks and splines)
ax0.show = False; ax1.show = False; ax2.show = False;
#ax3.show = False; ax4.show = False; ax5.show = False;

import toyplot.pdf
#toyplot.pdf.render(canvas, "/home/tim/GBS/Muscari/Mus_Analysis/Mus_tetRAD/tetRAD_Figures/Muscari_tetRAD_MajCon_pops30.pdf");

### Load and plot CP trees
#### load cp ML tree

In [4]:
Mus_CP_ML = toytree.tree("/home/tim/Phylo/Muscari_CP/Mus_CP_RAxML_20210806/RAxML_bipartitions.Mus_CP_ML_20210806")

rtre = Mus_CP_ML.root(wildcard = "Brim")

# use canvas and axes function in order use export function
canvas, axes, mark = rtre.ladderize(1).draw(
    width = 1400,
    height = 900,
    use_edge_length = False,
    tip_labels_align = True,
    node_labels='support',
    node_sizes=0,
    node_labels_style={"font-size": "16px",
                       "baseline-shift": "7px",
                       "-toyplot-anchor-shift": "-13px"},
    );

unrecognized arguments skipped: ['use_edge_length']
check the docs, argument names may have changed.


<svg class="toyplot-canvas-Canvas" height="900.0px" id="t886405139ac24d06ad5915f9a873d7e6" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 1400.0 900.0" width="1400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 71 64 76 71 16 90 61 86 17 60 97 69 82 28 57 91 91 100 99 100 88 53 40 90 93 22 53 31 54 100 70 23 47 96 48 14 99 36 100 100 89 94 98 81 99 88 100 90 85 100 100 100 100 100 Brimeura_amethystina_W6084_ Scilla_monophyll_ED1271_ Scilla_siberica_ED3235_ Bellevalia_paradoxa_ED1272_ Bellevalia_dubia_W6083_ Bellevalia_speciosa_W6085_ Muscari_parviflorum_ED1245_ Pseudomuscari_inconstrictum_ED3234_ Pseudomuscari_coeleste_ Pseudomuscari_chalusicum_ED1255_ Pseudomuscari_azureum_ED1270_ Muscari_racemosum_ED1258_ Muscari_massayanum_ED1251_ Muscari_macrocarpum_ED1252_ Muscari_commutatum_ED3538_ Muscari_mirum_ED1250_ Muscari_vuarlii_ED3232_ Muscari_anatolicum_W6087_ Muscari_anatolicum_ED1276_ Muscari_sivrihisardaghlarensis_ED1278_ Pseudomuscari_pallens_ED1267_ Pseudomuscari_coeruleum_ED1261_ Muscari_aucheri_ Muscari_armeniacum_ED1244_ Muscari_armeniacum_W6089_ Muscari_discolor_ED1266_ Muscari_macbeathianum_ Muscari_serpentinicum_ Muscari_sandrasicum_ Muscari_tuzgoluense_ Muscari_baeticum_ED1281_ Muscari_neglectum_ED1254_ Muscari_botryoides_ED1279_ Muscari_neglectum_ED1253_ Muscari_adilii_W6090_ Leopoldia_neumannii_HT_ED1243_ Leopoldia_neumannii_ED1607_ Muscari_pulchellum_ED3231_ Muscari_kerkis_ED1280_ Muscari_microstomum_ Muscari_latifolium_ED1265_ Muscari_bourgaei_ED1259_ Muscari_turcicum_ Leopoldia_longipes_ED3233_ Muscari_babachii_ Muscari_erdalii_ Leopoldia_spreitzenhoferii_ED1248_ Leopolida_cycladica_W6082_ Leopoldia_tenuiflora_ED1263_ Leopoldia_caucasica_ED1262_ Leopoldia_matritensis_ED1282_ Leopoldia_comosa_ED3539_ Leopoldia_comosa_ED3965_ Leopoldia_comosa_ED1274_ Leopoldia_weissii_W6081_ Leopoldia_comosa_ED1256_

In [6]:
Mus_CP_ML = toytree.tree("/home/tim/Phylo/Muscari_CP/Mus_CP_RAxML_20210806/RAxML_bipartitions.Mus_CP_ML_20210806")

rtre = Mus_CP_ML.root(wildcard = "Brim")

# use canvas and axes function in order use export function
canvas, axes, mark = rtre.ladderize(1).draw(
    width = 1400,
    height = 900,
    #use_edge_length = False,
    tip_labels_align = True,
    node_labels='support',
    node_sizes=0,
    node_labels_style={"font-size": "16px",
                       "baseline-shift": "7px",
                       "-toyplot-anchor-shift": "-13px"},
    );

<svg class="toyplot-canvas-Canvas" height="900.0px" id="t3b20faca45ff4fe5bff50820ad75fbc9" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 1400.0 900.0" width="1400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 69 64 75 77 15 89 91 66 18 59 97 83 76 30 57 91 92 100 100 88 100 56 38 89 92 23 53 30 56 100 70 25 50 94 51 15 98 39 100 100 88 94 97 80 99 89 100 89 85 100 100 100 100 100 Brimeura_amethystina_W6084_ Scilla_monophyll_ED1271_ Scilla_siberica_ED3235_ Bellevalia_paradoxa_ED1272_ Bellevalia_dubia_W6083_ Bellevalia_speciosa_W6085_ Muscari_parviflorum_ED1245_ Pseudomuscari_inconstrictum_ED3234_ Pseudomuscari_coeleste_ Pseudomuscari_chalusicum_ED1255_ Pseudomuscari_azureum_ED1270_ Muscari_racemosum_ED1258_ Muscari_macrocarpum_ED1252_ Muscari_massayanum_ED1251_ Muscari_mirum_ED1250_ Muscari_commutatum_ED3538_ Muscari_vuarlii_ED3232_ Muscari_anatolicum_W6087_ Muscari_anatolicum_ED1276_ Muscari_sivrihisardaghlarensis_ED1278_ Pseudomuscari_pallens_ED1267_ Pseudomuscari_coeruleum_ED1261_ Muscari_aucheri_ Muscari_armeniacum_ED1244_ Muscari_armeniacum_W6089_ Muscari_discolor_ED1266_ Muscari_macbeathianum_ Muscari_sandrasicum_ Muscari_serpentinicum_ Muscari_tuzgoluense_ Muscari_baeticum_ED1281_ Muscari_neglectum_ED1254_ Muscari_botryoides_ED1279_ Muscari_neglectum_ED1253_ Muscari_adilii_W6090_ Muscari_kerkis_ED1280_ Muscari_pulchellum_ED3231_ Leopoldia_neumannii_HT_ED1243_ Leopoldia_neumannii_ED1607_ Muscari_microstomum_ Muscari_latifolium_ED1265_ Muscari_bourgaei_ED1259_ Muscari_turcicum_ Leopoldia_longipes_ED3233_ Muscari_babachii_ Muscari_erdalii_ Leopoldia_comosa_ED3539_ Leopoldia_comosa_ED3965_ Leopoldia_comosa_ED1274_ Leopoldia_comosa_ED1256_ Leopoldia_weissii_W6081_ Leopolida_cycladica_W6082_ Leopoldia_spreitzenhoferii_ED1248_ Leopoldia_tenuiflora_ED1263_ Leopoldia_caucasica_ED1262_ Leopoldia_matritensis_ED1282_